This notebook aims to find near duplicates using the SimHash technique, and organizing them in an M-Tree

In [2]:
# import similarities.mtree
import importcsv
from similarities import libsim, mtree
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
print('Importing csv file...........')
small_dataset = importcsv.opencsv('processedspeechfinal.csv') 

Importing csv file...........


Κρατάω μόνο τα processed speeches

In [5]:
speeches = small_dataset['processed_speeches']
# speeches = small_dataset['speech']

In [4]:
print(speeches)
print(type(speeches[0]))
# speeches.head()

0         ΠΑΡΑΚΑΛΕΙΤΑ ΚΥΡΙ ΓΡΑΜΜΑΤΕ ΣΥΝΟΔΕΥΣ ΙΕΡ ΣΥΝΟΔ Α...
1         ΥΠΑΡΧ ΕΚ ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΨΗΦΙΣ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩΜ...
2         ΚΥΡΙ ΒΟΥΛΕΥΤ ΤΙΜ ΑΝΑΚΟΙΝΩΣ ΣΩΜ ΒΟΥΛΕΥΤ ΕΒΡ ΥΠΟ...
3         ΥΠΑΡΧ ΕΚ ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΕΨΗΦΙΣ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩ...
4         ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΤΙΜ ΑΝΑΚΟΙΝΩΣ ΑΠΟΤΕΛΕΣΜ ΔΙΕΞΑΧΘΕ...
                                ...                        
362625    Π ΤΕΘ ΠΕΡΙΟΡΙΣΜ ΠΑΜ ευρωπαϊκη ΕΝΩΣ ΖΗΤΑΜ ΛΕΜ ...
362626    ΥΠΗΡΧΕΕΜ ΑΝΑΠΤΥΞΙΑΚ ΥΠΟΥΡΓΕΙ ΥΠΟΥΡΓΕΙ ΚΑΝ ΚΟΙΝ...
362627    ΕΥΧΑΡΙΣΤΟΥΜ ΥΠΟΥΡΓ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩΜΕΝ ΣΥΖΗΤΗΣ...
362628    ΟΛΟΚΛΗΡΩΣ ΨΗΦΟΦΟΡΙ ΗΛΕΚΤΡΟΝΙΚ ΣΥΣΤΗΜ ΣΧΕΔΙ ΝΟΜ...
362629    ΣΥΝΑΙΝΕΣ ΣΩΜΑΤ ΩΡ 1949΄ ΛΥΕΤΑ ΣΥΝΕΔΡΙΑΣ ΔΕΥΤΕΡ...
Name: processed_speeches, Length: 362630, dtype: object
<class 'str'>


We need to define the weights of each word for all speeches, and keep them in a dictionary.
The dictionary is of the form {word: weight}

### Find word weights

In [6]:
# This shall come in handy
# https://www.reddit.com/r/LanguageTechnology/comments/dugjis/approximate_string_matching_with_simhash_and_a/
# def tokenizer(word):
#     return word.split()
# del vectorizer, X
# speeches = speeches.to_list()
vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=False, )
X = vectorizer.fit_transform(speeches.values)

In [7]:
names = vectorizer.get_feature_names_out()
index = 10550

print(type(names))


<class 'numpy.ndarray'>


We need word weights for this to work

In [8]:
max_idf = len(names) -1
weights = {name: (idx/max_idf) for idx, name in enumerate(names)} # normalized weights
weights

{'0000τοσο': 0.0,
 '0001ευχαριστω': 1.5693635132399353e-06,
 '0002γραφημα': 3.1387270264798705e-06,
 '0002θεωρουμε': 4.708090539719806e-06,
 '0003επιλεξει': 6.277454052959741e-06,
 '0004γων': 7.846817566199676e-06,
 '0006που': 9.416181079439611e-06,
 '0009ορισμενεσ': 1.0985544592679547e-05,
 '000αυτα': 1.2554908105919482e-05,
 '000δρχ': 1.4124271619159418e-05,
 '000καταστηματαρχεσ': 1.569363513239935e-05,
 '0010νομουσ': 1.726299864563929e-05,
 '0011δυο': 1.8832362158879222e-05,
 '0012αναφερθω': 2.040172567211916e-05,
 '0012βητηση': 2.1971089185359093e-05,
 '0013απο': 2.354045269859903e-05,
 '0013δεν': 2.5109816211838964e-05,
 '0014περιοριζετε': 2.6679179725078898e-05,
 '0016των': 2.8248543238318835e-05,
 '0018ξεφορτωνονται': 2.981790675155877e-05,
 '0019τονουσ': 3.13872702647987e-05,
 '001κυριε': 3.295663377803864e-05,
 '0021γιατι': 3.452599729127858e-05,
 '0021μπορουμε': 3.609536080451851e-05,
 '0021χωσ': 3.7664724317758445e-05,
 '0022μικρο': 3.9234087830998385e-05,
 '0022πρου': 4.080

Acquire the signature for one document

In [9]:
import importlib
importlib.reload(libsim)

sig = libsim.getSignatureOfSpeech(speeches[0], weights)
print('signature of first document:',bin(sig))

signature of first document: 0b1001011010111010010101001100100100000110101101000101010010111101


### Create M-Tree strucure
This data structure will hold all the signatures, based on their Hamming Distance.
This way, we can find the relevant ones

In [ ]:
# The following list is an array of signatures in integer form 
# signatures = [libsim.getSignatureOfSpeech(speech, weights) for speech in speeches]
signatures = {idx: libsim.getSignatureOfSpeech(speech, weights) for idx, speech in enumerate(speeches)}
# this took 14 minutes to complete

In [11]:
# run the following code? protection against Run-All
if 1 == 0: # change this if statement
    import pickle
    filename = 'speeches_signatures.pkl'
    with open(filename, 'wb') as fo:
        pickle.dump(signatures, fo)


In [17]:
# run the following code? protection against Run-All
if 1 == 0: # change this if statement
    import pickle
    filename = 'speeches_signatures.pkl'
    with open(filename, 'rb') as fo:
        signatures = pickle.load(fo) # this is a dictionary of {indexOfSpeech: signatureOfSpeech}

In [12]:
def hamming_distance(a: int, b:int):
    """
    This function returns the hamming distance between two integers.
    The hamming distance is defined as the total number of 1s, after
    the XOR operation between two binary numbers
    """
    return int.bit_count(a ^ b) # requires python 3.10

#### The magic number 3
If the distance between two document signatures is greater than 3, the documents are not near duplicates. This is taken as-is, no further investigation is needed. (Apostolos Papadopoulos)

In [18]:
tree = mtree.MTree(hamming_distance, max_node_size=10)
tree.add_all(signatures)
# takes 13 seconds to build the tree

In [14]:
if 1 == 0: # protection against Run-All operations
    import pickle
    with open('sim_tree.pkl', 'wb') as fo:
        pickle.dump(tree,fo)

In [25]:
m = tree.search(signatures[1], 4) # this returns a map object

Let's see if a query is possible

In [26]:
list(m)

[111603, 308211, 111601, 275443]

### We have a tree of signatures
Now all the signatures are stored in a dictionary, indexed based on speech